In [ ]:
import tensorflow as tf
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import math


def hyperParamGen():
    #random sampes using numpy
    logLearningRate = np.random.uniform(-7.5, -4.5)
    print("logLearningRate", logLearningRate)
    learningRate = np.exp(logLearningRate)
    print("learningRate", learningRate)
    numLayers = np.random.random_integers(1, 5)
    print("numLayers", numLayers)
    numHiddenUnits = np.random.random_integers(100, 500)
    print("numHiddenUnits", numHiddenUnits)
    logWeightedDecay = np.random.uniform(-9, -6)
    print("logWeightedDecay", logWeightedDecay)
    weightedDecay = np.exp(logWeightedDecay)
    print("weightedDecay", weightedDecay)
    dropOut = np.random.random_integers(0, 1) # 0 or 1
    print("dropOut", dropOut)
    
    return learningRate, numLayers, numHiddenUnits, weightedDecay, dropOut

def convertTarget(targetValues):
    numClasses = np.max(targetValues) + 1
    return np.eye(numClasses)[targetValues]

class loadData:
    def __init__(self):
        self.flatten = True
        self.addOnes = False
        
        self.data_path = '/Users/vikuo/Documents/GitHub/ece521/assi/A3/notMNIST.npz'
    def arrFlatten(self, arr):
        '''
        type np array
        '''
        dataDim1, dum1, dum2 = arr.shape
        dum_sq = dum1 * dum2
        arr = np.reshape(arr, [ dataDim1 ,dum_sq ])
        return arr   
    '''
    def convertTarget(self, targetValues):
        numClasses = np.max(targetValues) + 1
        return np.eye(numClasses)[targetValues]
    '''
    def loadNumData(self):
        with np.load(self.data_path) as data:
            Data, Target = data ["images"], data["labels"]
            np.random.seed(521)
            randIndx = np.arange(len(Data))
            np.random.shuffle(randIndx)
            Data = Data[randIndx]/255.
            
            if self.flatten:
                Data = self.arrFlatten(Data)
            
            Target = Target[randIndx]
            trainData, trainTarget = Data[:15000], Target[:15000]
            validData, validTarget = Data[15000:16000], Target[15000:16000]
            testData, testTarget = Data[16000:], Target[16000:]
            
            trainTarget = convertTarget(trainTarget)
            validTarget = convertTarget(validTarget)
            testTarget = convertTarget(testTarget)
 
        return trainData, trainTarget, validData, validTarget, testData, testTarget

class BatchSampler(object):
    '''
    A (very) simple wrapper to randomly sample batches without replacement.
    '''
    
    def __init__(self, data, targets, batch_size):
        self.num_points = data.shape[0]
        self.features = data.shape[1]
        self.data = data
        self.targets = targets
        self.batch_size = batch_size
        self.indices = np.arange(self.num_points)

    def random_batch_indices(self, m=None):
        if m is None:
            indices = np.random.choice(self.indices, self.batch_size, replace=False)
        else:
            indices = np.random.choice(self.indices, m, replace=False)
        return indices 

    def get_batch(self, m=None):
        '''
        Get a random batch without replacement from the dataset.
        If m is given the batch will be of size m. 
        Otherwise will default to the class initialized value.
        '''
        indices = self.random_batch_indices(m)
        X_batch = np.take(self.data, indices, 0)
        y_batch = self.targets[indices]
        return X_batch, y_batch


class neuralNetwork:
    # these are for testing only
    def __init__(self, _learningRate = 0.05, _learningRateArr = [0.05], 
                 _numLayers = [1000], _weightedDecay = 3e-4, _dropOut = 0, _epochIter = 10000):
        # learningRate, numLayers, numHiddenUnits, weightedDecay, dropOut
        self.learningRate = _learningRate
        self.learningRateArr = _learningRateArr
        self.numLayers = _numLayers # number of units in each layer
        self.numLayers.append(10)
        print("init layer", self.numLayers)
        self.weightedDecay = _weightedDecay
        self.dropOut = _dropOut
        self.dropOutProb = 0.5
        self.epochIter = _epochIter 
        
        
        # default, no arg taken
        self.numPixel = 784 
        self.numClass = 10
        self.batchSize = 500
        
    def buildLayer(self, _inputTensor, _numUnits):
        '''
        input:
            #feed# _inputTensor S = theta(Xprev)from the prev layer
            _numUnits is the num of neurons in this layer
    
        intermediate xavierInit:
            W is initialized as Xavier
            W is input.shape[1] by num units
    
        output:
        weighted sum of inputs
    
        '''
        # zero mean independent Gaussians whose variance is 3/(#input + #outputs)  
        dim1 = _inputTensor.get_shape().as_list()[1]
        

        
        #Xavier
        #W = tf.get_variable("W", shape = [dim1, _numUnits], initializer=tf.contrib.layers.xavier_initializer())
        variance = 3.0 / (dim1 + _numUnits)
        W = tf.Variable(tf.truncated_normal(shape = [dim1, _numUnits],  stddev = math.sqrt(variance)))
        #b = tf.Variable(0.0, name='biases')
        b = tf.Variable(tf.zeros([_numUnits]), name = 'biases')

        
        print("W", dim1, _numUnits)
        print("X shape", _inputTensor.get_shape().as_list())
        X = tf.matmul(tf.cast(_inputTensor, dtype = tf.float32), W) + b
    
        #X = tf.add(X, b)
        print("X in layer", X)
        regTerm = tf.multiply( tf.constant(0.50, dtype = tf.float32),\
                        tf.multiply(tf.constant(self.weightedDecay,dtype = tf.float32), tf.reduce_mean(tf.square(W))))
        
        tf.add_to_collection("layerW", W)
        tf.add_to_collection("layerReg", regTerm)

        return X
    
    
        
        
    def accuracy(self, y_hat, target):
        #TAKEN FROM last assignment
        
        target = tf.cast(target, dtype = tf.float32)
        correctCases = tf.equal(tf.argmax(y_hat, 1), tf.argmax(target, 1))
        accuracy = tf.reduce_mean(tf.cast(correctCases, dtype="float"))
        return accuracy #.eval()
    
    def classificationError(self, y_hat, target):
        target = tf.cast(target, dtype = tf.float32)
        correctCases = tf.equal(tf.argmax(y_hat, 1), tf.argmax(target, 1))
        
        error = 1 - tf.reduce_mean(tf.cast(correctCases, dtype="float"))
        return error

    
    def buildNet(self):
        '''
        input: 
            number of hidden units #in the class def
            data #feed
        output:
            predicted labels
        '''
                
        X = tf.placeholder(tf.float32, shape=[None, self.numPixel], name='dataX')
        y_target = tf.placeholder(tf.float32, shape=[None, self.numClass], name='targetY')

        X_prev = X #tf.convert_to_tensor(X)
        for numUnits in self.numLayers: #numLayers is an array of num hidden units            
            X_curr = self.buildLayer(X_prev, numUnits)
            print("X_curr", X_curr)
            S_curr = tf.nn.relu(X_curr)
            X_prev = S_curr
            if self.dropOut:
                X_prev = tf.nn.dropout(X_prev, self.dropOutProb)
        
        y_hat = X_curr
        
        crossEntropyErrorCurr = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits = y_hat, labels = y_target))
        
        regTermSum = sum(tf.get_collection("regTerm"))
        regTermSum = tf.cast(regTermSum, dtype = tf.float32)
        print("regTermsum", regTermSum)
        print("crossEntropyError Curr", crossEntropyErrorCurr)
        
        totalLoss = tf.add(regTermSum , crossEntropyErrorCurr)
        
        optimizer = tf.train.AdamOptimizer(learning_rate = self.learningRate)
        train = optimizer.minimize(loss=totalLoss)
        accuracyTerm = self.accuracy(y_hat, y_target)

        W = tf.get_collection("layerW")
        
        # return totalLoss also??
        return W, X, y_hat, y_target, crossEntropyErrorCurr, accuracyTerm, train   #final prediction
    
        
        
        
    def plotFig(self, _dim, y , addInfo, title="default", xLabel="iter/epoch", yLabel="yLabel", plotLabel ="plotLabel", _num =1 ):
        x = np.linspace(0, _dim, num=_dim)
        y = np.array(y)
        print("$$$$$$$$$$$$$$$ in plot fig$$$$$$$$$$$$$$$$$")
        print(y.shape)
        plt.figure(_num)
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        for i in range(y.shape[0]):
            #print("x", x)
            #print("y", y[i])
            plt.plot(x, y[i], label = plotLabel + str(addInfo[i]))
        
        plt.legend()
        plt.savefig( title + str(_num) + ".png")
        plt.close()
        plt.clf()
        
    def runPart1_1(self, trainData, trainTarget, validData, validTarget,testData, testTarget):
        '''
        input: data
                3 learning rates to try
        output:
            classification error plot
            cross entropy loss plot
        '''
        range_list = [0.25, 0.5, 0.75, 1] # save at progress %
        trainLossArr= []
        mseLossArr = []
        trainAccuArr = []
        
        validLossArr = []
        validAccuArr = []
        
        
        for learningRate in self.learningRateArr:
            currEpoch = 0


            
            trainLossL = []
            mseLossL = []
            trainAccuL = []
            
            validLossL =[]
            validAccuL = []
            
            tf.reset_default_graph()

            W, X, y_hat, y_target, crossEntropyErrorCurr, accuracyTerm, train = self.buildNet()
            
            init = tf.global_variables_initializer()
            saver = tf.train.Saver()

            sess = tf.InteractiveSession()
            sess.run(init)
            #initialW = sess.run(W)  
            print("learningrate = ", learningRate)
            trainBatchSampler = BatchSampler(trainData, trainTarget, self.batchSize)
            print("self.epochIter", self.epochIter)

            for i in range (0, self.epochIter):
                dataBatch, targetBatch = trainBatchSampler.get_batch()
                currentW, errTrain, accuracyTrain, y_predict, trainModel = \
                        sess.run([W, crossEntropyErrorCurr, accuracyTerm, y_hat, train], feed_dict={X: dataBatch, y_target: targetBatch})
                
                for j in range_list:
                    if i == int(j * self.epochIter):
                        print("currEpoch", i ,"accuracyTrain" , accuracyTrain, "errTrain", errTrain)
                        saver.save(sess, './my-model', global_step=i)
                    
        # plot at 25%, 50%, 75%, 100%
        #fileName = 
        
def restoreSaver():
    
        restore_saver = tf.train.import_meta_graph('my_model-1000.meta')
        restore_saver.restore(sess,tf.train.latest_checkpoint('./'))
        print("trying tf saver")
        print(sess.run(crossEntropyErrorCurr))

        

    
    




if __name__ == '__main__':  

    dataLoader = loadData()
 
    trainData, trainTarget, validData, validTarget,testData, testTarget = dataLoader.loadNumData()
    
    print("*******q1 part 1********")
    learningRates = [0.001]#[0.001, 0.001, 0.1] #for debugging accuracy
    lambdaReg = 3e-4
    # TODO: Plotting

    singleLayerNet = neuralNetwork(_learningRateArr=learningRates, _epochIter = trainData.shape[0])

    singleLayerNet.runPart1_1(trainData, trainTarget, validData, validTarget,testData, testTarget)
    #restoreSaver()
    

    
                      


    

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


*******q1 part 1********
init layer [1000, 10]
W 784 1000
X shape [None, 784]
X in layer Tensor("add:0", shape=(?, 1000), dtype=float32)
X_curr Tensor("add:0", shape=(?, 1000), dtype=float32)
W 1000 10
X shape [None, 1000]
X in layer Tensor("add_1:0", shape=(?, 10), dtype=float32)
X_curr Tensor("add_1:0", shape=(?, 10), dtype=float32)
regTermsum Tensor("Cast_3:0", shape=(), dtype=float32)
crossEntropyError Curr Tensor("Mean_2:0", shape=(), dtype=float32)
learningrate =  0.001
self.epochIter 15000
currEpoch 3750 accuracyTrain 0.944 errTrain 0.22418


# TO DO:
* iterations over epoch
